We've heard from users that they often use Jupyter for prototyping, and then the move to SQL files for deployment. We could automate that step.

To enable this, we run:

```python
%config SqlMagic.persist_snippets = True
```

In [1]:
%load_ext sql

In [2]:
%sql duckdb://

In [3]:
%%sql --save penguins
SELECT
  *
FROM
  penguins.csv

Running query in 'duckdb://'

(duckdb.IOException) IO Error: No files found that match the pattern "penguins.csv"
[SQL: SELECT
  *
FROM
  penguins.csv]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

With the `persist_snippets` option turned on, `--save penguins` will store the snippet in `jupysql-snippets/penguins.sql`

And running this:

In [12]:
%%sql --save penguins_male
SELECT
  *
FROM
  penguins
WHERE
  sex = 'MALE'

Generating CTE with stored snippets : penguins


Running query in 'duckdb://'

species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
Adelie,Torgersen,39.1,18.7,181,3750,MALE
Adelie,Torgersen,39.3,20.6,190,3650,MALE
Adelie,Torgersen,39.2,19.6,195,4675,MALE
Adelie,Torgersen,38.6,21.2,191,3800,MALE
Adelie,Torgersen,34.6,21.1,198,4400,MALE
Adelie,Torgersen,42.5,20.7,197,4500,MALE
Adelie,Torgersen,46.0,21.5,194,4200,MALE
Adelie,Biscoe,37.7,18.7,180,3600,MALE
Adelie,Biscoe,38.2,18.1,185,3950,MALE
Adelie,Biscoe,38.8,17.2,180,3800,MALE


Should store the generated CTE in `jupysql-snippets/penguins_male.sql`

In [8]:
cte = %sqlrender penguins_male
print(cte)

WITH penguins AS (
SELECT
  *
FROM
  penguins.csv)
SELECT
  *
FROM
  penguins
where sex = 'MALE'


But when loading it from the `.sql` file, only the:

```sql
SELECT
  *
FROM
  penguins
WHERE
  sex = 'MALE'
```

part should be assigned to the `penguins_male` snippet.

When initializing the magic, we should automatically load the snippets. i.e., if I start a new notebook, and there is a `jupysql-snippets/` directory, the snippets should load:

In [13]:
%sqlcmd snippets

'penguins, penguins_male'

Important limitation: this first implementation would only allow one-way editing. For example, if I go and edit the `.sql` file, the changes won't be reflected when I open the notebook. So we must be clear to the users that they *should not* edit `.sql` files by hand, but edit it from the notebook. In the next iteration we can work on two-way syncing.